In [1]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model=deployment_name,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
        )

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Define a tool that gets the weather for a city.
async def get_weather(city: str) -> str:
    """Get the weather for a city."""
    return f"The weather in {city} is 72 degrees and Sunny."


# Create an assistant agent.
weather_agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    tools=[get_weather],
    system_message="Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition.
text_termination = TextMentionTermination("TERMINATE")

# Create a single-agent team.
single_agent_team = RoundRobinGroupChat([weather_agent], termination_condition=text_termination)

async def run_team() -> None:
    result = await single_agent_team.run(task="What is the weather in New York?")
    print(result)


# Use `asyncio.run(run_team())` when running in a script.
await run_team()


messages=[TextMessage(id='69b4bb8a-1ea5-4814-85c2-fafb9f1f4878', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 41, 12, 274924, tzinfo=datetime.timezone.utc), content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(id='c95ebd2a-6826-407e-8b23-359d54330b27', source='assistant', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=16), metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 41, 13, 241265, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_NwCoFNl1vmHsaL9phRtsoWtw', arguments='{"city":"New York"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='0717fe2b-dc12-4a05-9976-2e558a93c210', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 41, 13, 242238, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', name='get_weather', call_id='

In [3]:
from autogen_agentchat.base import TaskResult


async def run_team_stream() -> None:
    async for message in single_agent_team.run_stream(task="What is the weather in New York?"):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)


# Use `asyncio.run(run_team_stream())` when running in a script.
await run_team_stream()



id='b52213e3-6303-4e18-ab37-79ee3ba6dba6' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 15, 5, 41, 36, 138224, tzinfo=datetime.timezone.utc) content='What is the weather in New York?' type='TextMessage'
id='d6ad608d-1138-48f0-b8ae-73e5d8d0d0fc' source='assistant' models_usage=RequestUsage(prompt_tokens=594, completion_tokens=16) metadata={} created_at=datetime.datetime(2025, 7, 15, 5, 41, 36, 843192, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='call_inCIeguzk4FyMbhmudyqo2Qt', arguments='{"city":"New York"}', name='get_weather')] type='ToolCallRequestEvent'
id='250782db-1ef7-4547-8d0c-f28ff93ba5f4' source='assistant' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 15, 5, 41, 36, 845504, tzinfo=datetime.timezone.utc) content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', name='get_weather', call_id='call_inCIeguzk4FyMbhmudyqo2Qt', is_error=False)] type='ToolCallExecutionEvent'
id='434

In [4]:
from autogen_agentchat.ui import Console

# Use `asyncio.run(single_agent_team.reset())` when running in a script.
await single_agent_team.reset()  # Reset the team for the next run.
# Use `asyncio.run(single_agent_team.run_stream(task="What is the weather in Seattle?"))` when running in a script.
await Console(
    single_agent_team.run_stream(task="What is the weather in Seattle?")
)  # Stream the messages to the console.

---------- TextMessage (user) ----------
What is the weather in Seattle?
---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='call_9rkrFTMoXlhVz3q0ph9AuSEb', arguments='{"city":"Seattle"}', name='get_weather')]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', name='get_weather', call_id='call_9rkrFTMoXlhVz3q0ph9AuSEb', is_error=False)]
---------- ToolCallSummaryMessage (assistant) ----------
The weather in Seattle is 72 degrees and Sunny.
---------- TextMessage (assistant) ----------
The weather in Seattle is currently 72 degrees and sunny.
---------- TextMessage (assistant) ----------
The current weather in Seattle is 72 degrees and sunny.
---------- TextMessage (assistant) ----------
The weather in Seattle is currently 72 degrees and sunny.
---------- TextMessage (assistant) ----------
The current weather in Seattle is 72 degrees and sunny.
---------- TextMessage (assistant

TaskResult(messages=[TextMessage(id='0a001986-9991-44a5-8c3b-137f6456d4c9', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 41, 48, 612123, tzinfo=datetime.timezone.utc), content='What is the weather in Seattle?', type='TextMessage'), ToolCallRequestEvent(id='9c05d252-2c1c-48e9-af9e-eb04848a9661', source='assistant', models_usage=RequestUsage(prompt_tokens=69, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 41, 49, 199258, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_9rkrFTMoXlhVz3q0ph9AuSEb', arguments='{"city":"Seattle"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='bcf3d72d-b737-4652-b298-d01717b6ff2b', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 41, 49, 200861, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', name='get_weather', c

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(5)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Create a team with the primary and critic agents.
reflection_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=termination)

In [6]:
# Use `asyncio.run(Console(reflection_team.run_stream(task="Write a short poem about fall season.")))` when running in a script.
await Console(
    reflection_team.run_stream(task="Write a short poem about fall season.")
)  # Stream the messages to the console.

---------- TextMessage (user) ----------
Write a short poem about fall season.
---------- TextMessage (primary) ----------
Golden leaves drift to the ground,  
Whispering secrets without a sound.  
Crisp air carries autumn’s song,  
A fleeting dance before winter’s strong.
---------- TextMessage (critic) ----------
Your poem beautifully captures the essence of fall with vivid imagery and a gentle rhythm. The phrase "Whispering secrets without a sound" adds a touch of mystery, perfectly complementing the quiet, reflective atmosphere of autumn. To enhance clarity, consider rephrasing the last line; "winter's strong" feels a bit ambiguous and might benefit from a more specific descriptor, such as "winter's dawn" or "winter's chill." Overall, it's a lovely and evocative piece!
---------- TextMessage (primary) ----------
Thank you for your thoughtful feedback! I appreciate your suggestion to clarify the last line. Here's a revised version with that adjustment:

Golden leaves drift to the gr

TaskResult(messages=[TextMessage(id='1d65f925-fe0b-4612-8942-7cb105e128a7', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 42, 20, 787033, tzinfo=datetime.timezone.utc), content='Write a short poem about fall season.', type='TextMessage'), TextMessage(id='a0a3846b-b5ea-4517-9600-e012c153a618', source='primary', models_usage=RequestUsage(prompt_tokens=27, completion_tokens=36), metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 42, 21, 703433, tzinfo=datetime.timezone.utc), content='Golden leaves drift to the ground,  \nWhispering secrets without a sound.  \nCrisp air carries autumn’s song,  \nA fleeting dance before winter’s strong.', type='TextMessage'), TextMessage(id='234c9559-a146-4fde-a5b4-b5aa68d6ffc7', source='critic', models_usage=RequestUsage(prompt_tokens=80, completion_tokens=98), metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 42, 23, 231544, tzinfo=datetime.timezone.utc), content='Your poem beautifully captures 

In [7]:
await Console(reflection_team.run_stream(task="중국 당나라 시풍으로 시를 써보세요."))

---------- TextMessage (user) ----------
중국 당나라 시풍으로 시를 써보세요.
---------- TextMessage (primary) ----------
물론입니다. 당나라 시풍은 자연과 인간의 조화를 중시하며, 간결하면서도 깊은 의미를 담는 것이 특징입니다. 다음은 중국 당나라 시풍으로 쓴 가을을 주제로 한 시입니다.

가을 산중향 (秋山中郷)  

단풍 붉게 물들고  
청천 끝없이 높네  
한 줄기 바람 스치면  
낙엽 춤추네 고요히  

(해석)  
가을 산에 붉게 물든 단풍,  
푸른 하늘은 끝없이 높구나.  
바람 한 줄기 스치면,  
고요히 낙엽이 춤춘다.  

간결한 구절 속에 자연의 변화와 고요한 정취를 담아 보았습니다. 도움이 되셨길 바랍니다!
---------- TextMessage (critic) ----------
APPROVE


TaskResult(messages=[TextMessage(id='7737f914-eb07-4ade-a359-a2070de4de76', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 42, 28, 533375, tzinfo=datetime.timezone.utc), content='중국 당나라 시풍으로 시를 써보세요.', type='TextMessage'), TextMessage(id='2cf5fd41-27ca-4671-aa18-ca96148bc08a', source='primary', models_usage=RequestUsage(prompt_tokens=273, completion_tokens=185), metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 42, 31, 65232, tzinfo=datetime.timezone.utc), content='물론입니다. 당나라 시풍은 자연과 인간의 조화를 중시하며, 간결하면서도 깊은 의미를 담는 것이 특징입니다. 다음은 중국 당나라 시풍으로 쓴 가을을 주제로 한 시입니다.\n\n가을 산중향 (秋山中郷)  \n\n단풍 붉게 물들고  \n청천 끝없이 높네  \n한 줄기 바람 스치면  \n낙엽 춤추네 고요히  \n\n(해석)  \n가을 산에 붉게 물든 단풍,  \n푸른 하늘은 끝없이 높구나.  \n바람 한 줄기 스치면,  \n고요히 낙엽이 춤춘다.  \n\n간결한 구절 속에 자연의 변화와 고요한 정취를 담아 보았습니다. 도움이 되셨길 바랍니다!', type='TextMessage'), TextMessage(id='43f6d9d8-8185-4286-a36f-f63b43764fba', source='critic', models_usage=RequestUsage(prompt_tokens=475, completion_tokens=4), metadata={},

In [8]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    system_message="Always transfer to user when you don't know the answer. Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")
termination = handoff_termination | text_termination

# Create a single-agent team.
lazy_agent_team = RoundRobinGroupChat([lazy_agent], termination_condition=termination)

In [ ]:
from autogen_agentchat.ui import Console

await Console(lazy_agent_team.run_stream(task="서울의 기후는 어때?"))

---------- TextMessage (user) ----------
서울의 기후는 어때?
---------- ToolCallRequestEvent (lazy_assistant) ----------
[FunctionCall(id='call_91tYRV1o6wXDIkImjjqbhycg', arguments='{}', name='transfer_to_user')]
---------- ToolCallExecutionEvent (lazy_assistant) ----------
[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_91tYRV1o6wXDIkImjjqbhycg', is_error=False)]
---------- HandoffMessage (lazy_assistant) ----------
Transfer to user.


TaskResult(messages=[TextMessage(id='93a8c195-9cb9-4df8-8d94-50479a35f5e8', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 42, 37, 309322, tzinfo=datetime.timezone.utc), content='서울의 기후는 어때?', type='TextMessage'), ToolCallRequestEvent(id='ccda57e9-ffcf-44d7-b626-f2206a52bb23', source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=68, completion_tokens=12), metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 42, 37, 984792, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_91tYRV1o6wXDIkImjjqbhycg', arguments='{}', name='transfer_to_user')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='162e3e26-e632-4353-b95e-368f102574d1', source='lazy_assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 42, 37, 985383, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_91tYRV1o6wXDIkImjjqbhycg', is_err